In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [0]:
# 自行上传训练和预测文件，俩csv。下面是预训练模型所需词典，模型和配置文件
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-tf_model.h5
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json

--2020-04-15 15:05:26--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-tf_model.h5
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.233.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.233.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 478309336 (456M) [binary/octet-stream]
Saving to: ‘bert-base-chinese-tf_model.h5’

bert-base-chinese-t 100%[===================>] 456.15M  32.8MB/s    in 16s     

2020-04-15 15:05:43 (28.2 MB/s) - ‘bert-base-chinese-tf_model.h5’ saved [478309336/478309336]



In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
!pip install transformers
from transformers import *

     |████████████████████████████████| 573kB 8.8MB/s 
     |████████████████████████████████| 1.0MB 61.3MB/s 
     |████████████████████████████████| 890kB 59.5MB/s 
     |████████████████████████████████| 3.7MB 44.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=65c1ae9ce4c59e28bb6d80813ae0767ef791183fc0e4be218cf49efbb1149f87
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
PATH = './'
BERT_PATH = './'
MAX_SEQUENCE_LENGTH = 128
input_categories = '微博中文内容'
output_categories = '情感倾向'

'''if not os.path.exists(BERT_PATH):
    os.mkdir(BERT_PATH)
'''
df_train = pd.read_csv(PATH+'nCoV_100k_train.labled.csv',engine ='python',encoding='utf-8')
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]
df_test = pd.read_csv(PATH+'nCov_10k_test.csv',engine ='python',encoding='utf-8')
df_sub = pd.read_csv(PATH+'submit_example.csv',encoding='utf-8')

In [0]:
def _convert_to_transformer_inputs(instance, tokenizer, max_sequence_length):

  def return_id(str1, truncation_strategy, length):
    inputs = tokenizer.encode_plus(str1,add_special_tokens=True,max_length=length,truncation_strategy=truncation_strategy)
    input_ids = inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]
  
  input_ids, input_masks, input_segments = return_id(instance, 'longest_first', max_sequence_length)

  return [input_ids, input_masks, input_segments]

In [0]:
def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
  input_ids, input_masks, input_segments = [], [], []
  for instance in tqdm(df[columns]):
      ids, masks, segments = \
          _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)

      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32),
          np.asarray(input_masks, dtype=np.int32),
          np.asarray(input_segments, dtype=np.int32)
          ]

def compute_output_arrays(df:pd.DataFrame, columns:str) -> np.ndarray:
  return np.asarray(df[columns].astype(int)+1)

In [0]:
def create_model():
  input_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
  input_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
  input_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
  config = BertConfig.from_pretrained(BERT_PATH + 'bert-base-chinese-config.json')
  config.output_hidden_states = False
  bert_model = TFBertModel.from_pretrained(BERT_PATH + 'bert-base-chinese-tf_model.h5', config=config)
  # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
  embedding = bert_model(input_id, attention_mask=input_mask, token_type_ids=input_atn)[0]

  x = tf.keras.layers.GlobalAveragePooling1D()(embedding)
  x = tf.keras.layers.Dropout(0.15)(x)
  x = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.models.Model(inputs=[input_id, input_mask, input_atn], outputs=x)

  return model

In [10]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-chinese-vocab.txt')

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [11]:
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

100%|██████████| 99913/99913 [00:56<00:00, 1781.91it/s]


In [12]:
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

100%|██████████| 10000/10000 [00:05<00:00, 1778.84it/s]


In [0]:
outputs = compute_output_arrays(df_train, output_categories)
gkf = StratifiedKFold(n_splits=5).split(X=df_train[input_categories].fillna('-1'),y=df_train[output_categories].fillna('-1'))
valid_preds = []
test_preds = []

In [22]:
for fold, (train_idx, valid_idx) in enumerate(gkf):
  train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
  train_outputs = to_categorical(outputs[train_idx])

  valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
  valid_outputs = to_categorical(outputs[valid_idx])

  K.clear_session()
  model = create_model()
  optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc', 'mae'])
  # 注意batch_size不要太大，16是使用 colab Tesla T4 运行，一次大约15分钟，自己运行注意设置batch参数，太大则GPU运行不了
  model.fit(train_inputs, train_outputs, validation_data=[valid_inputs, valid_outputs], epochs=1, batch_size=16)
  # model.save_weights(f'bert-{fold}.h5')
  valid_preds.append(model.predict(valid_inputs))
  test_preds.append(model.predict(test_inputs))
  # 想用交叉验证，把下面的break注释
  break

4996/4996 [==============================] - 1282s 257ms/step - loss: 0.6099 - acc: 0.7326 - mae: 0.2352 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00 - val_mae: 0.0000e+00


In [0]:
# 保存结果
sub = np.average(test_preds, axis=0)
sub = np.argmax(sub, axis=1)
df_sub['y'] = sub - 1
df_sub['id'] = df_sub['id'].apply(lambda x: str(x) + ' ')
df_sub.to_csv('我为疫情献计策-final.csv', index=False, encoding='utf-8')